In [57]:
from datetime import datetime, timedelta
import numpy as np
from ordered_set import OrderedSet
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, ConstantVelocity
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState

np.random.seed(1992)

num_steps = 50
pos_range = 50
vel_range = 1
birth_rate = 0.1  # Adjust the birth rate as needed

start_time = datetime.now().replace(microsecond=0)
truths = OrderedSet()  # Truths across all time
current_truths = set() # Truths alive at the current time

# Define a transition model with some noise
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(0.05), ConstantVelocity(0.05)])

# Calculate the FOV center
fov_center = [pos_range / 2, pos_range / 2]

# Function to calculate inward velocity vector towards FOV center
def inward_velocity_vector(position):
    # Calculate the vector pointing from the position to the FOV center
    direction_vector = [fov_center[0] - position[0], fov_center[1] - position[1]]
    # Normalize the direction vector
    norm = np.linalg.norm(direction_vector)
    if norm == 0:
        return [0, 0]  # Handle the case where position is already at the FOV center
    return [direction_vector[0] / norm * vel_range, direction_vector[1] / norm * vel_range]


# Function to generate random position at the edges of the FOV
def random_edge_position():
    edge = np.random.choice([0, 1, 2, 3])  # 0: left, 1: right, 2: top, 3: bottom
    if edge == 0:
        return [0, np.random.uniform(0, pos_range)]
    elif edge == 1:
        return [pos_range, np.random.uniform(0, pos_range)]
    elif edge == 2:
        return [np.random.uniform(0, pos_range), pos_range]
    else:
        return [np.random.uniform(0, pos_range), 0]





timesteps = []
for k in range(num_steps):
    timesteps.append(start_time + timedelta(seconds=k))

    # Death (Only for truths within FOV)
    for truth in current_truths.copy():
        if (truth[-1].state_vector[0] < 0 or truth[-1].state_vector[0] > pos_range or
            truth[-1].state_vector[2] < 0 or truth[-1].state_vector[2] > pos_range):
            current_truths.remove(truth)

    # Update truths
    for truth in current_truths:
        truth.append(GroundTruthState(transition_model.function(truth[-1], noise=True, time_interval=timedelta(seconds=1)), timestamp=timesteps[k]))

    # Birth based on Poisson process
    num_births = np.random.poisson(birth_rate)
    for _ in range(num_births):
        initial_position = random_edge_position()
        initial_velocity = inward_velocity_vector(initial_position)  # Inward velocity
        state = GroundTruthState([initial_position[0], initial_velocity[0], initial_position[1], initial_velocity[1]], timestamp=timesteps[k])
        truth = GroundTruthPath([state])
        current_truths.add(truth)
        truths.add(truth)




In [58]:
#from stonesoup.plotter import AnimatedPlotterly
#plotter = AnimatedPlotterly(timesteps, tail_length=1)
#plotter.plot_ground_truths(truths, [0, 2])
#plotter.fig

In [59]:
from scipy.stats import uniform
from stonesoup.types.detection import TrueDetection
from stonesoup.types.detection import Clutter
from stonesoup.models.measurement.linear import LinearGaussian

clutter_rate = 1.5

measurement_model = LinearGaussian(
    ndim_state=4,
    mapping=(0, 2),
    noise_covar=np.array([[0.25, 0],
                          [0, 0.25]])
    )
all_measurements = []

for k in range(num_steps):
    measurement_set = set()
    timestamp = start_time + timedelta(seconds=k)

    for truth in truths:
        try:
            truth_state = truth[timestamp]
        except IndexError:
            # This truth not alive at this time.
            continue
        # Generate actual detection from the state with a 10% chance that no detection is received.
        if np.random.rand() <= 0.9:
            # Generate actual detection from the state
            measurement = measurement_model.function(truth_state, noise=True)
            measurement_set.add(TrueDetection(state_vector=measurement,
                                              groundtruth_path=truth,
                                              timestamp=truth_state.timestamp,
                                              measurement_model=measurement_model))

        # Generate clutter at this time-step
        truth_x = truth_state.state_vector[0]
        truth_y = truth_state.state_vector[2]
        num_clutter = np.random.poisson(clutter_rate)
        for _ in range(num_clutter):
            x = uniform.rvs(truth_x - 10, 20)
            y = uniform.rvs(truth_y - 10, 20)
            measurement_set.add(Clutter(np.array([[x], [y]]), timestamp=timestamp,
                                        measurement_model=measurement_model))
    all_measurements.append(measurement_set)

# Plot true detections and clutter.
#plotter.plot_measurements(all_measurements, [0, 2])
#plotter.fig

In [60]:
from stonesoup.predictor.kalman import KalmanPredictor
predictor = KalmanPredictor(transition_model)

from stonesoup.updater.kalman import KalmanUpdater
updater = KalmanUpdater(measurement_model)

from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.measures import Mahalanobis
hypothesiser = DistanceHypothesiser(predictor, updater, measure=Mahalanobis(), missed_distance=3)

from stonesoup.dataassociator.neighbour import GNNWith2DAssignment
data_associator = GNNWith2DAssignment(hypothesiser)

from stonesoup.deleter.error import CovarianceBasedDeleter
deleter = CovarianceBasedDeleter(covar_trace_thresh=3)

from stonesoup.types.state import GaussianState
from stonesoup.initiator.simple import MultiMeasurementInitiator
initiator = MultiMeasurementInitiator(
    prior_state=GaussianState([[0], [0], [0], [0]], np.diag([0, 1, 0, 1])),
    measurement_model=measurement_model,
    deleter=deleter,
    data_associator=data_associator,
    updater=updater,
    min_points=5,
    )

In [61]:
tracks, all_tracks = set(), set()

for n, measurements in enumerate(all_measurements):
    # Calculate all hypothesis pairs and associate the elements in the best subset to the tracks.
    hypotheses = data_associator.associate(tracks,
                                           measurements,
                                           start_time + timedelta(seconds=n))
    associated_measurements = set()
    for track in tracks:
        hypothesis = hypotheses[track]
        if hypothesis.measurement:
            post = updater.update(hypothesis)
            track.append(post)
            associated_measurements.add(hypothesis.measurement)
        else:  # When data associator says no detections are good enough, we'll keep the prediction
            track.append(hypothesis.prediction)

    # Carry out deletion and initiation
    tracks -= deleter.delete_tracks(tracks)
    tracks |= initiator.initiate(measurements - associated_measurements,
                                 start_time + timedelta(seconds=n))
    all_tracks |= tracks

In [62]:
#plotter.plot_tracks(all_tracks, [0, 2], uncertainty=True)
#plotter.fig

In [63]:
import matplotlib.pyplot as plt
from stonesoup.plotter import AnimatedPlotterly
import plotly.graph_objects as go

# Define the CustomAnimatedPlotter class
class CustomAnimatedPlotter(AnimatedPlotterly):
    def __init__(self, timesteps, tail_length=1):
        super().__init__(timesteps, tail_length=tail_length)

    def plot_field_of_view(self, fov_coords):
        # Unpack FOV coordinates
        x_min, x_max, y_min, y_max = fov_coords

        # Draw the FOV rectangle
        fov_trace = go.Scatter(
            x=[x_min, x_max, x_max, x_min, x_min],
            y=[y_min, y_min, y_max, y_max, y_min],
            mode='lines',
            fill='toself',
            fillcolor='rgba(255, 0, 0, 0.2)',  # Red with opacity
            line=dict(color='red', width=2),
            name='Field of View (FOV)'
        )

        # Create a Plotly figure for the FOV
        fig = go.Figure(data=[fov_trace])

        # Set axis limits based on FOV coordinates
        fig.update_xaxes(range=[x_min - 10, x_max + 10])
        fig.update_yaxes(range=[y_min - 10, y_max + 10])

        # Show the FOV plot
        fig.show()

# Instantiate the custom plotter
plotter = CustomAnimatedPlotter(timesteps, tail_length=1)

# Plot ground truths, measurements, tracks, etc.
plotter.plot_ground_truths(truths, [0, 2])
plotter.plot_measurements(all_measurements, [0, 2])
plotter.plot_tracks(all_tracks, [0, 2], uncertainty=True)

# Plot the field of view using Plotly
plotter.plot_field_of_view(fov_coords)
plotter.fig
